In [ ]:
import PIL
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
import os 
import tqdm
import glob
import tensorflow 
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.color import *

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import load_img, img_to_array

physical_devices = tensorflow.config.list_physical_devices("GPU")
tensorflow.config.experimental.set_memory_growth(physical_devices[0], True)
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.2)

valid_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.2)

test_datagen  = ImageDataGenerator(rescale = 1./255)
train_dataset  = train_datagen.flow_from_directory(directory = 'OASISDataset/OASISDataset',
                                                   target_size = (224,224),
                                                   class_mode = 'categorical',
                                                   subset = 'training',
                                                   batch_size = 128)

valid_dataset = valid_datagen.flow_from_directory(directory = 'OASISDataset/OASISDataset',
                                                  target_size = (224,224),
                                                  class_mode = 'categorical',
                                                  subset = 'validation',
                                                  batch_size = 128)
# Model Initialization

base_model = MobileNetV2(input_shape=(224,224,3), 
                         include_top=False,
                         weights="imagenet")
# Freezing Layers

for layer in base_model.layers:
    layer.trainable=False
# Building Model

model=Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1024,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4,activation='softmax'))

# Summary
model.summary()

# Model Compile 
OPT    = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(loss='categorical_crossentropy',
              metrics=['accuracy', tensorflow.keras.metrics.AUC(name = 'auc')],
              optimizer=OPT)
# Defining Callbacks

filepath = './best_weights.hdf5'

earlystopping = EarlyStopping(monitor = 'val_auc', 
                              mode = 'max' , 
                              patience = 15,
                              verbose = 1)

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_auc', 
                                mode='max', 
                                save_best_only=True, 
                                verbose = 1)


callback_list = [earlystopping, checkpoint]
model_history = model.fit(train_dataset,
                          validation_data=valid_dataset,
                          epochs=20, 
                          callbacks=callback_list,
                          verbose=1)

In [ ]:
#Test Data 
test_dataset = test_datagen.flow_from_directory(directory='OASISDataset/OASISDataset',
                                                target_size=(224, 224),
                                                class_mode='categorical',
                                                batch_size=128)
# Evaluating Loss and AUC
evaluation_results = model.evaluate(test_dataset)
print(f'Test Loss: {evaluation_results[0]}, Test Accuracy: {evaluation_results[1]}, Test AUC: {evaluation_results[2]}')

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import seaborn as sns

# Custom class names
class_names = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']  # Update with your actual class names

# Test Data 
test_dataset = test_datagen.flow_from_directory(directory='OASISDataset/OASISDataset',
                                                target_size=(224, 224),
                                                class_mode='categorical',
                                                batch_size=128,
                                                shuffle=False)  # Ensure predictions match true labels

# Get predictions from the model
predictions = model.predict(test_dataset)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels
true_labels = test_dataset.classes

# Calculate confusion matrix
cm = confusion_matrix(true_labels, predicted_classes)

# Plot confusion matrix with custom class names
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# Calculate accuracy for each class
class_accuracy = {}
for i, class_name in enumerate(class_names):
    true_positives = cm[i, i]
    total_samples = np.sum(cm[i, :])
    class_accuracy[class_name] = true_positives / total_samples

# Print accuracy for each class
for class_name, acc in class_accuracy.items():
    print(f'Accuracy for {class_name}: {acc:.2f}')

# Classification Report
report = classification_report(true_labels, predicted_classes, target_names=class_names)
print("\nClassification Report:")
print(report)

model.save('TrainedMObilenetv2.keras')
model.save_weights('TrainedMobilenetv2Weights.h5')